    This strategy, checked with multi time frame and different indicators
    are used. And checked with stocks from Nifty 50
    Last update Dec 2021

In [1]:
import pandas as pd
import numpy as np
from time import sleep
import datetime
import talib
import indicators

In [2]:
def read_data_backetesting(path, name, time_frame, start_date, end_date):
    df = pd.read_csv(path+time_frame+'/'+name+'.csv')
    df = df.set_index(df['date'])
    df = df[start_date:end_date]
    return df

def apply_indicators(df, ema_small, ema_big, ema_bigger, rsi_param, atr_param, ):
    df['masmall'] = talib.EMA(df['close'], timeperiod = ema_small)
    df['mabig'] = talib.EMA(df['close'], timeperiod = ema_big)
    df['mabigger'] = talib.EMA(df['close'], timeperiod = ema_bigger)
    df['rsi'] = talib.RSI(df['close'], timeperiod = rsi_param)
    df['atr'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod = atr_param)
    df['adx'] = talib.ADX(df['high'], df['low'], df['close'])
    df['previous_candle_close'] = df['close'].shift(1)
    indicators.SuperTrend(df=df, period=7, multiplier=3, ohlc=['open', 'high', 'low', 'close'])
    return df

def check_entry_condition(index, candle1, df3, df5, df15, dfday, rsi_bull_level, rsi_bear_level):
    empty_series = {'masmall':0, 'mabig':0, 'mabigger':0, 'rsi':0, 'atr':0, 'adx':0, 'previous_candle_close':0, 
                    'STX_7_3':0, 'close':0}
    candle3 = pd.Series(empty_series)
    candle5 = pd.Series(empty_series)
    candle15 = pd.Series(empty_series)
    if (int(dtime[14:16]) % 3 == 0):
#         print('Checking actual 3min candle')
        candle3 = df3.loc[dtime]
#         print(candle3)
    if (int(dtime[14:16]) % 5 == 0):
        candle5 = df5.loc[dtime]
    if (int(dtime[14:16]) % 15 == 0):
        candle15 = df15.loc[dtime]
        
    # buy condition on 3 min timeframe
    buy_condition1 = candle3['masmall'] > candle3['mabig'] > candle3['mabigger']
    buy_condition2 = candle3['rsi'] > rsi_bull_level
    buy_condition3 = candle3['STX_7_3'] == 'up'
    buy_condition4 = candle3['adx'] > 25
#     buy_condition4 = True
    buy_condition5 = candle3['close'] > candle3['previous_candle_close']
    buy_condition6 = candle3['close'] > dfday.loc[index[:11] + '00:00:00+05:30']['yh']
    
    # buy condition on 5 min timeframe
    buy_condition7 = candle5['masmall'] > candle5['mabig'] > candle5['mabigger']
    buy_condition8 = candle5['rsi'] > rsi_bull_level
    buy_condition9 = candle5['STX_7_3'] == 'up'
    buy_condition10 = candle5['adx'] > 22.5
#     buy_condition10 = True
    buy_condition11 = candle5['close'] > candle5['previous_candle_close']
    buy_condition12 = candle5['close'] > dfday.loc[index[:11] + '00:00:00+05:30']['yh']
    
    # buy condition on 15 min timeframe
    buy_condition13 = candle15['masmall'] > candle15['mabig'] > candle15['mabigger']
    buy_condition14 = candle15['STX_7_3'] == 'up'
    buy_condition15 = candle15['adx'] > 19
#     buy_condition15 = True
    buy_condition16 = candle15['close'] > candle15['previous_candle_close']
    
    return buy_condition1, buy_condition2, buy_condition3, buy_condition4, buy_condition5, buy_condition6, buy_condition7, buy_condition8, buy_condition9, buy_condition10, buy_condition11, buy_condition12, buy_condition13, buy_condition14, buy_condition15, buy_condition16

In [3]:
# watchlist = ['NIFTY BANK', 'NIFTY 50']
watchlist = ['ADANIPORTS', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 
            'BAJFINANCE',  'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA','DIVISLAB','DRREDDY',
             'GRASIM','HCLTECH','HDFC','HDFCBANK','HEROMOTOCO','HINDALCO','HINDUNILVR','ICICIBANK','INDUSINDBK',
             'INFY','IOC','ITC','JSWSTEEL','KOTAKBANK','LT','M&M','MARUTI','NESTLEIND','NTPC','ONGC','POWERGRID',
             'RELIANCE','SBILIFE','SBIN','SHREECEM','SUNPHARMA','TATACONSUM','TATAMOTORS','TATASTEEL','TCS',
             'TECHM','TITAN','ULTRACEMCO','UPL','WIPRO']
path = './Historical_saved_data/'

In [4]:
status = {'name':None, 'date':None, 'entry_time': None, 'entry_price': None, 'buy_sell': None, 'qty':None, 
          'target':None, 'sl':None, 'exit_time': None, 'exit_price':None, 'pnl':None, 'remark' : None, 'traded' : None}
risk_per_trade = 1000
final_results = {}
tradeno = 0

In [5]:
# required_date = '2020-06-01 09:15:00+05:30'
start_date = '2021-01-01'
end_date = '2021-06-24'
ema_small = 8
ema_big = 20
ema_bigger = 50
rsi_param = 7
atr_param = 7
rsi_bull_level = 60
rsi_bear_level = 40
atr_multiplier = 0.5*2

In [9]:
for name in watchlist[:10]:
    status = {'name':None, 'date':None, 'entry_time': None, 'entry_price': None, 'buy_sell': None, 'qty':None, 
          'target':None, 'sl':None, 'exit_time': None, 'exit_price':None, 'pnl':None, 'remark' : None, 'traded' : None}
    total_pl = 0
    total_sl, profitable_trades = 0, 0
    total_investment = 0
    print('Backtesting for: ', name)
    
    # Read 1, 3, 5, 15 minutes and daily data
    df1 = read_data_backetesting(path, name, 'minute', start_date, end_date)
    df3 = read_data_backetesting(path, name, '3minute', start_date, end_date)
    df5 = read_data_backetesting(path, name, '5minute', start_date, end_date)
    df15 = read_data_backetesting(path, name, '15minute', start_date, end_date)
    dfday = read_data_backetesting(path, name, 'day', start_date, end_date)
    
    # apply indicators to required data 
    df1 = apply_indicators(df1, ema_small, ema_big, ema_bigger, rsi_param, atr_param)
    df3 = apply_indicators(df3, ema_small, ema_big, ema_bigger, rsi_param, atr_param)
    df5 = apply_indicators(df5, ema_small, ema_big, ema_bigger, rsi_param, atr_param)
    df15 = apply_indicators(df15, ema_small, ema_big, ema_bigger, rsi_param, atr_param)
    dfday = apply_indicators(dfday, ema_small, ema_big, ema_bigger, rsi_param, atr_param)
    
    dfday['yh'] = dfday['high'].shift(1)
    dfday['yl'] = dfday['low'].shift(1)
    
    try:
        for dtime, candle in df1[:].iterrows():
            entry_values = check_entry_condition(dtime, candle, df3, df5, df15, dfday, rsi_bull_level, rsi_bear_level)
            bc1, bc2, bc3, bc4, bc5, bc6, bc7, bc8, bc9, bc10, bc11, bc12, bc13, bc14, bc15, bc16 = entry_values

            # Entry condition
            if bc1 and bc2 and bc3 and bc4 and bc5 and bc6 and bc7 and bc8 and bc9 and bc10 and bc11 and bc12 and bc13 and bc14 and bc15 and bc16:
                tradeno += 1
                status['name'] = name
                status['date'] = dtime[:10] 
                status['entry_time'] = dtime[:16]
                status['entry_price'] = candle['close']
                status['buy_sell'] = 'buy'
                status['sl'] = round((candle['close'] - (atr_multiplier*candle['atr'])), 1)
                status['target'] = candle['close'] * 1.01
                status['qty'] = 10
                status['traded'] = 'yes'
                total_investment += status['entry_price'] * status['qty']
#                 print(f"Buy {name}, time {dtime[:16]}, price {candle['close']}")
                continue

            # Exit condition
            if status['traded'] == 'yes':

                if (status['buy_sell'] == 'buy') and ((candle['low'] < status['sl']) or (dtime[11:16] == "15:15") or (candle['high'] >= status['target'])):

                    # if target is achieved
                    if (candle['high'] >= status['target']):
                        status['pnl'] = round((status['target'] - status['entry_price'])*status['qty'] , 2)
                        status['remark'] = 'buy_target'
                        status['exit_price'] = status['target']
                        status['exit_time'] = dtime[:16]
                        total_pl += ((status['target'] - status['entry_price']))*status['qty']
#                         print(f"Sell at Target {name}, time {dtime[:16]}, price {status['target']}")
#                         print(f"Profit: {round((status['target'] - status['entry_price'])*status['qty'] , 2)}")

                    # if sl is hit
                    if (candle['low'] <= status['sl']):
                        status['pnl'] = round(((status['sl'] - status['entry_price']))*status['qty'], 2)
                        status['remark'] = 'buy_sl_hit'
                        status['exit_price'] = status['sl']
                        status['exit_time'] = dtime[:16]
                        total_pl += ((status['sl'] - status['entry_price']))*status['qty']
#                         print(f"Sell at SL {name}, time {dtime[:16]}, price {status['sl']}")
#                         print('Loss: ', round(((status['sl'] - status['entry_price']))*status['qty'], 2))
                        total_sl += 1

                    # if no sl hit, then exit at 15:15
                    if (dtime[11:16] == '15:15'):
                        status['pnl'] = round(((candle['close'] - status['entry_price']))*status['qty'], 2)
                        status['remark'] = 'market_over'
                        status['exit_price'] = candle['close']
                        status['exit_time'] = dtime[:16]
                        total_pl += ((candle['close'] - status['entry_price']))*status['qty']
#                         print(f"Sell at Market close {name}, time {dtime[:16]}, price {candle['close']}")
#                         print('Profit: ', round(((candle['close'] - status['entry_price']))*status['qty'], 2))

                    final_results[tradeno] = status
                    status = {'name':None, 'date':None, 'entry_time': None, 'entry_price': None, 'buy_sell': None, 'qty':None, 
              'target':None, 'sl':None, 'exit_time': None, 'exit_price':None, 'pnl':None, 'remark' : None, 'traded' : None}
    
    except Exception as e:
        print('Error: ', e)
        
    print('Total Trades: ', tradeno)
    print('Total SL: ', total_sl, 'SL trade percentage: ', round((total_sl/tradeno)*100 , 2))
    print('Total P&L: ', round(total_pl, 2))
    print('Total investment: ', total_investment)
    print('Profit percentage: ', round((total_pl / total_investment) * 100, 2))
    
    # reset status
    status = {'name':None, 'date':None, 'entry_time': None, 'entry_price': None, 'buy_sell': None, 'qty':None, 

              'target':None, 'sl':None, 'exit_time': None, 'exit_price':None, 'pnl':None, 'remark' : None, 'traded' : None}

    pd.DataFrame(final_results).T.to_csv('./BT_results/bt3.1/'+name+'.csv', index = None)
    final_results = {}
    tradeno = 0
    print('############## ***** ##############')

Backtesting for:  ADANIPORTS
Total Trades:  92
Total SL:  42 SL trade percentage:  45.65
Total P&L:  1553.71
Total investment:  643527.0
Profit percentage:  0.24
############## ***** ##############
Backtesting for:  ASIANPAINT
Total Trades:  54
Total SL:  32 SL trade percentage:  59.26
Total P&L:  -6.55
Total investment:  1443278.0
Profit percentage:  -0.0
############## ***** ##############
Backtesting for:  AXISBANK
Total Trades:  56
Total SL:  26 SL trade percentage:  46.43
Total P&L:  674.3
Total investment:  398895.5
Profit percentage:  0.17
############## ***** ##############
Backtesting for:  BAJAJ-AUTO
Total Trades:  50
Total SL:  28 SL trade percentage:  56.0
Total P&L:  1449.33
Total investment:  1931549.0
Profit percentage:  0.08
############## ***** ##############
Backtesting for:  BAJAJFINSV
Total Trades:  68
Total SL:  33 SL trade percentage:  48.53
Total P&L:  13615.02
Total investment:  7019803.5
Profit percentage:  0.19
############## ***** ##############
Backtesting f